<a href="https://colab.research.google.com/github/dfx-44/--my-config-files/blob/branch/martin_cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.functional as F

import torch
from torchvision import datasets
import torchvision.transforms as transforms

from torch.nn import functional as F



device = ("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])


# traning set download
train_set = datasets.CIFAR10(root='./data',download=True,transform=transform,train=True)

# test set download
test_set = datasets.CIFAR10(root='./data',download=True,transform=transform,train=False)



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
trainloader = torch.utils.data.DataLoader(train_set,batch_size=4,shuffle=True,num_workers=2)

validloader = torch.utils.data.DataLoader(test_set,batch_size=4,shuffle=False,num_workers=2)

In [ ]:
print("the shape of the images is: ",test_set[0][0].shape)

the shape of the images is:  torch.Size([3, 32, 32])


In [ ]:
from torch.nn.modules.container import Sequential
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # first conv layers
        self.network  = nn.Sequential( 

        # TODO: make it with 6 conv layers along   with relu and max_p and use batchnorm2d
        # and three linear layers on the end and do a flatten before pass to linear layers

            #  first layer
            nn.Conv2d(3, 32, 3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(32),

            # second layer
            nn.Conv2d(32, 64, 3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) ,
            nn.BatchNorm2d(64),


            # third layer
            nn.Conv2d(64, 70, 3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) ,
            nn.BatchNorm2d(70),

            # fourth layer
            nn.Conv2d(70,16, 3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(16),

        # flatten goes here: 
            nn.Flatten(),

            # fifth layer 
            nn.Linear(64,50),
            nn.Linear(50, 10)
        
            )
    
    # it's the forward function that defines the network structure
    # we're accepting only a single input in here, but if you want,
    # feel free to use more
    def forward(self, x):

        x = self.network(x)

        # in your model definition you can go full crazy and use arbitrary
        # python code to define your model structure
        # all these are perfectly legal, and will be handled correctly
        # by autograd:
        # if x.gt(0) > x.numel() / 2:
        #      ...
        #
        # you can even do a loop and reuse the same module inside it
        # modules no longer hold ephemeral state, so you can use them
        # multiple times during your forward pass
        # while x.norm(2) < 10:
        #    x = self.conv1(x)

        return x 

model = Net()

In [ ]:

if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
epochs = 20
train_loss, val_loss = [], []
accuracy_total_train, accuracy_total_val = [], []
criterion = nn.CrossEntropyLoss()
for epoch in range(epochs):
   
    total_train_loss = 0
    total_val_loss = 0

    model.train()
    
    total = 0
    # training our model
    for idx, (image, label) in enumerate(trainloader):
            
        image, label = image.to(device), label.to(device)

        optimizer.zero_grad()

        pred = model(image)

        loss = criterion(pred, label)
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

        pred = torch.nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
                
    accuracy_train = total / len(train_set)
    accuracy_total_train.append(accuracy_train)

    total_train_loss = total_train_loss / (idx + 1)
    train_loss.append(total_train_loss)
    
    # validating our model
    model.eval()
    total = 0
    for idx, (image, label) in enumerate(validloader):
        image, label = image.cuda(), label.cuda()
        pred = model(image)
        loss = criterion(pred, label)
        total_val_loss += loss.item()

        pred = torch.nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1

    accuracy_val = total / len(test_set)
    accuracy_total_val.append(accuracy_val)

    total_val_loss = total_val_loss / (idx + 1)
    val_loss.append(total_val_loss)

    print("Epoch: {}/{}  ".format(epoch, epochs),
        "Training loss: {:.4f}  ".format(total_train_loss),
        "Testing loss: {:.4f}  ".format(total_val_loss),
        "Train accuracy: {:.4f}  ".format(accuracy_train),
        "Test accuracy: {:.4f}  ".format(accuracy_val))




Epoch: 0/20   Training loss: 1.5187   Testing loss: 1.2016   Train accuracy: 0.4622   Test accuracy: 0.5684  
Epoch: 1/20   Training loss: 1.1305   Testing loss: 1.0368   Train accuracy: 0.6025   Test accuracy: 0.6392  
Epoch: 2/20   Training loss: 0.9895   Testing loss: 0.9856   Train accuracy: 0.6527   Test accuracy: 0.6592  
Epoch: 3/20   Training loss: 0.8950   Testing loss: 0.9243   Train accuracy: 0.6873   Test accuracy: 0.6738  
Epoch: 4/20   Training loss: 0.8244   Testing loss: 0.8645   Train accuracy: 0.7134   Test accuracy: 0.7041  
Epoch: 5/20   Training loss: 0.7689   Testing loss: 0.8563   Train accuracy: 0.7322   Test accuracy: 0.7037  
Epoch: 6/20   Training loss: 0.7263   Testing loss: 0.8291   Train accuracy: 0.7462   Test accuracy: 0.7129  
Epoch: 7/20   Training loss: 0.6871   Testing loss: 0.8201   Train accuracy: 0.7598   Test accuracy: 0.7174  
Epoch: 8/20   Training loss: 0.6533   Testing loss: 0.8103   Train accuracy: 0.7706   Test accuracy: 0.7239  
Epoch: 9/2